In [1]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-sciencefair69420')

Exception: Problem requesting tokens. Please try again.  HTTP Error 400: Bad Request b'{\n  "error": "invalid_request",\n  "error_description": "Missing required parameter: code"\n}'

In [2]:
def dataset_value(dataset, lon, lat):
    point = ee.Geometry.Point([lon, lat])
    if isinstance(dataset, ee.imagecollection.ImageCollection):
        dataset = dataset.mean()
    value = dataset.reduceRegion(
        reducer=ee.Reducer.first(), 
        geometry=point, 
        scale=30
    ).getInfo()
    return value

In [3]:
elevation = ee.Image("NASA/NASADEM_HGT/001").select('elevation')
temperature = ee.ImageCollection("MODIS/061/MOD11A1").select('LST_Day_1km')
biomass = ee.ImageCollection("WCMC/biomass_carbon_density/v1_0").select('carbon_tonnes_per_ha')
fires = ee.FeatureCollection("USFS/GTAC/MTBS/burned_area_boundaries/v1")

In [12]:
blackest = [1, 0]

for i in range(2, 69420):
    fire_feature = ee.Feature(fires.toList(i).get(i-1))
    
    # Get latitude and longitude of the fire feature
    lat = float(fire_feature.get('BurnBndLat').getInfo())
    lon = float(fire_feature.get('BurnBndLon').getInfo())

    # Check if the fire feature is within California's geographical boundaries
    if 32 <= lat <= 42 and -124 <= lon <= -114:
        ans = fire_feature.get('BurnBndAc').getInfo()
        if ans > blackest[1]:
            blackest = [i, ans]
    
    print(i, ans)


2 8385
3 4872
4 19441
5 710
6 880
7 1897
8 5615
9 1011
10 1280
11 606
12 1097
13 12790
14 8405
15 1337
16 2341
17 619
18 808
19 893
20 739
21 1853
22 2024
23 1655
24 1020
25 3181
26 1062
27 3536
28 2288
29 1492
30 1179
31 1406
32 588
33 1603
34 793
35 1570
36 19376
37 5556
38 778
39 545
40 702
41 2150
42 1591
43 2614
44 1641
45 20764
46 1526
47 5541
48 5206
49 1048
50 1752
51 707
52 622
53 1068
54 1019
55 3980
56 2173
57 1239
58 2352
59 954
60 5098
61 1383
62 1303
63 3624
64 1057
65 2059
66 727
67 1391
68 2304
69 2234
70 1468
71 763
72 25861
73 2828
74 4086
75 3098
76 2446
77 1060
78 1323
79 1454
80 5030
81 1154
82 1262
83 10824
84 1779
85 823
86 2481
87 1678
88 5107
89 1296
90 15296
91 13093
92 1365
93 4727
94 2645
95 1342
96 11733
97 1152
98 48001
99 8972
100 1291
101 2240
102 4495
103 1009
104 3360
105 1091
106 148188
107 1004
108 1445
109 13004
110 1582
111 3475
112 1250
113 5792
114 4249
115 1311
116 1713
117 863
118 2081
119 1233
120 2222
121 11027
122 1330
123 4687
124 3518
125 

KeyboardInterrupt: 

In [13]:
print(blackest)

[1737, 438772]


In [14]:
n=1737
fire_feature = ee.Feature(fires.toList(n).get(n-1))
fire_bounds = fire_feature.geometry().bounds()
buffer_distance = 1

# Create a rectangle around the fire perimeter with a buffer
rectangle = fire_bounds.buffer(buffer_distance).bounds()

In [15]:
def generate_grid(corners, point_spacing):
    coords = []
    min_lon, min_lat = corners[0]
    max_lon, max_lat = corners[1]

    lon = min_lon
    while lon < max_lon:
        lat = min_lat
        while lat < max_lat:
            coords.append([lon, lat])
            lat += point_spacing
        lon += point_spacing
    return coords


In [23]:
# Define the spacing between points (in degrees)
point_spacing = 0.01  # Example: approximately 111 meters at the equator

# Extract the lower-left and upper-right corners of the rectangle
rect_coords = rectangle.bounds().getInfo()['coordinates'][0]
lower_left = rect_coords[0]  # Lower-left corner
upper_right = rect_coords[2]  # Upper-right corner
    
grid_points = generate_grid([lower_left, upper_right], point_spacing)
print(lower_left, upper_right)
center = [0, 0]
center[0] = (lower_left[0]+upper_right[0])/2
center[1] = (lower_left[1]+upper_right[1])/2
print(center)

[-143.22445891920736, 67.33246597523797] [-141.39237195573972, 67.75243326261675]
[-142.30841543747354, 67.54244961892735]


In [24]:
import geemap.foliumap as geemap
import folium
Map = geemap.Map()
for point in grid_points:
    color = 'blue'
    if fire_feature.geometry().contains(ee.Geometry.Point(point)).getInfo():
        color = 'red'
    folium.Circle(
        location=point[::-1],
        radius=10,
        color=color
    ).add_to(Map)
Map

KeyboardInterrupt: 

In [28]:
data = []
for lon, lat in grid_points:
    elev = dataset_value(elevation, lon, lat)
    temp = dataset_value(temperature, lon, lat)
    bio = dataset_value(biomass, lon, lat)
    burned = fire_feature.geometry().contains(ee.Geometry.Point([lon, lat])).getInfo()
    data.append([elev['elevation'], temp['LST_Day_1km'], bio['carbon_tonnes_per_ha'], burned])

KeyboardInterrupt: 

In [32]:
import csv

# Open a new CSV file
with open('output.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # Write each row of the array to the CSV
    for row in data:
        writer.writerow(row)

In [13]:
import csv

# Initialize an empty list to hold the 2D array
datapp = []

# Open the CSV file
with open('output.csv', newline='') as csvfile:
    # Create a CSV reader object
    reader = csv.reader(csvfile)

    # Iterate over each row in the CSV file
    for row in reader:
        # Append the row to your data list
        datapp.append(row)

# Now, 'data' is a 2D array containing your CSV file data
print(datapp)

[['870', '14590.814870635519', '18.355554580688477', 'False'], ['853', '14604.709556057185', '54.5513916015625', 'False'], ['831', '14604.709556057185', '54.5513916015625', 'False'], ['799', '14604.709556057185', '54.5513916015625', 'False'], ['774', '14604.709556057185', '57.01499938964844', 'False'], ['755', '14604.709556057185', '57.01499938964844', 'False'], ['722', '14615.632765531062', '57.01499938964844', 'False'], ['694', '14615.632765531062', '44.41539001464844', 'False'], ['693', '14615.632765531062', '44.41539001464844', 'False'], ['672', '14764.49231970841', '44.41539001464844', 'False'], ['667', '14764.49231970841', '37.37649917602539', 'False'], ['677', '14764.49231970841', '37.37649917602539', 'False'], ['676', '14696.615306639635', '37.37649917602539', 'False'], ['681', '14696.615306639635', '36.60222244262695', 'False'], ['682', '14696.615306639635', '36.60222244262695', 'False'], ['677', '14696.615306639635', '62.01260757446289', 'False'], ['693', '14696.615306639635'

In [14]:
for i in range(len(datapp)):
    lon, lat = grid_points[i]
    datapp[i] += [(lon-center[0])**2+(lat-center[1])**2]
print(datapp)

[['870', '14590.814870635519', '18.355554580688477', 'False', 0.005801927901027936], ['853', '14604.709556057185', '54.5513916015625', 'False', 0.005719074296861195], ['831', '14604.709556057185', '54.5513916015625', 'False', 0.005638220692694443], ['799', '14604.709556057185', '54.5513916015625', 'False', 0.0055593670885276825], ['774', '14604.709556057185', '57.01499938964844', 'False', 0.005482513484360912], ['755', '14604.709556057185', '57.01499938964844', 'False', 0.005407659880194132], ['722', '14615.632765531062', '57.01499938964844', 'False', 0.005334806276027344], ['694', '14615.632765531062', '44.41539001464844', 'False', 0.005263952671860546], ['693', '14615.632765531062', '44.41539001464844', 'False', 0.0051950990676937385], ['672', '14764.49231970841', '44.41539001464844', 'False', 0.005128245463526922], ['667', '14764.49231970841', '37.37649917602539', 'False', 0.005063391859360096], ['677', '14764.49231970841', '37.37649917602539', 'False', 0.00500053825519326], ['676',

In [15]:
import csv

# Open a new CSV file
with open('outputpp.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # Write each row of the array to the CSV
    for row in datapp:
        writer.writerow(row)